In [8]:
import torch
from train import Config , MoENullModel


In [6]:
ckpt = torch.load(
    "checkpoints/checkpoint_3000.pt",
    map_location="cpu",
    weights_only=False
)

print(type(ckpt))
print(ckpt.keys())


<class 'dict'>
dict_keys(['step', 'model_state_dict', 'optimizer_state_dict', 'config'])


In [7]:
ckpt

{'step': 3000,
 'model_state_dict': OrderedDict([('rope_cos',
               tensor([[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
                       [ 0.5403,  0.7318,  0.8460,  ...,  1.0000,  1.0000,  1.0000],
                       [-0.4161,  0.0709,  0.4315,  ...,  1.0000,  1.0000,  1.0000],
                       ...,
                       [ 0.7877,  0.8723,  0.3831,  ...,  0.9996,  0.9998,  0.9999],
                       [ 0.9440,  0.9716, -0.1684,  ...,  0.9996,  0.9997,  0.9999],
                       [ 0.2324,  0.5497, -0.6680,  ...,  0.9995,  0.9997,  0.9999]])),
              ('rope_sin',
               tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
                         0.0000e+00,  0.0000e+00],
                       [ 8.4147e-01,  6.8156e-01,  5.3317e-01,  ...,  2.3714e-04,
                         1.7783e-04,  1.3335e-04],
                       [ 9.0930e-01,  9.9748e-01,  9.0213e-01,  ...,  4.7427e-04,
                      

In [9]:
cfg = ckpt["config"]

model = MoENullModel(cfg)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()

print("Model loaded")


Model loaded


In [ ]:
vocab_size = cfg.vocab_size
seq_len = 16   # keep this small

#“Give me a 2D tensor of shape (1, seq_len)
#where each element is an integer from 0 to vocab_size-1”

dummy_input = torch.randint(
    0, vocab_size, (1, seq_len), dtype=torch.long
)

with torch.no_grad():
    out = model(dummy_input)

print("Output shape:", out.shape)


Output shape: torch.Size([1, 16, 50257])


In [15]:
dummy_input = torch.randint(
    0,
    vocab_size,
    (1, 16),          # batch=1, seq=16
    dtype=torch.long
)

torch.onnx.export(
    model,
    (dummy_input,),
    "model.onnx",
    opset_version=17,
    input_names=["input_ids"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch", 1: "seq_len"},
        "logits":    {0: "batch", 1: "seq_len"},
    },
    dynamo=False,  # legacy exporter, doesn't use torch.export at all
)

print("ONNX export done")


/var/folders/_w/2kflckcx4js79vxh4vk8sq480000gn/T/ipykernel_46261/3053163252.py:8: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter has become the default. Learn more about the new export logic: https://docs.pytorch.org/docs/stable/onnx_export.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html
  torch.onnx.export(
/Users/amananand/Desktop/moe-null/train.py:83: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert D%2 == 0
/Users/amananand/Desktop/moe-null/train.py:519: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python value

ONNX export done
